<a href="https://colab.research.google.com/github/AshishGusain17/via_google_colab/blob/master/clr_snapshot_combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# example of saving models for a snapshot ensemble
from sklearn.datasets import make_blobs
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import Callback
from keras.optimizers import SGD
from keras import backend
from math import pi
from math import cos
from math import floor

from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import *
import numpy as np




from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt



import matplotlib
matplotlib.use("Agg")

In [10]:
# custom callback class for cyclic LR
class CyclicLR(Callback):

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())


In [11]:
# snapshot ensemble class
class SnapshotEnsemble(Callback):
	# constructor
	def __init__(self, n_epochs, n_cycles, lrate_max, verbose=0):
		self.epochs = n_epochs
		self.cycles = n_cycles
		self.lr_max = lrate_max
		self.lrates = list()

	# save models at the end of each cycle
	def on_epoch_end(self, epoch, logs={}):
		# check if we can save model
		epochs_per_cycle = floor(self.epochs / self.cycles)
		if epoch != 0 and (epoch + 1) % epochs_per_cycle == 0:
			# save model to file
			filename = "snapshot_model_%d.h5" % int((epoch + 1) / epochs_per_cycle)
			self.model.save(filename)
			print('>saved snapshot %s, epoch %d' % (filename, epoch))


In [12]:
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float")
testX = testX.astype("float")
print(trainX.shape , testX.shape)

# apply mean subtraction to the data
mean = np.mean(trainX, axis=0)
trainX -= mean
testX -= mean

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)
print(trainY.shape , testY.shape)



# construct the image generator for data augmentation
aug = ImageDataGenerator(width_shift_range=0.1,
	height_shift_range=0.1, horizontal_flip=True,
	fill_mode="nearest")


[INFO] loading CIFAR-10 data...
(50000, 32, 32, 3) (10000, 32, 32, 3)
(50000, 10) (10000, 10)


In [16]:
ct = 0
import keras



m1 = keras.applications.ResNet50(include_top=False,
                  input_shape = (32,32,3),
                  weights = 'imagenet')

model = keras.models.Sequential() 
model.add(m1)

for layer in model.layers:
  ct=ct+1

for layer in model.layers:
  if ct>10:
    layer.trainable = False
  else:
    layer.trainable = True
  ct=ct-1


model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()



/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten_3 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                20490     
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________


In [17]:
model.compile(loss="categorical_crossentropy", 
              optimizer='adam',
	          metrics=["accuracy"])


In [18]:

# initialize the cyclical learning rate callback
print("[INFO] using '{}' method".format("triangular"))
clr = CyclicLR(
	mode = "triangular",
	base_lr = 1e-7,
	max_lr = 1e-2,
	step_size = 8 * (trainX.shape[0] // 64))



# callback for snapshot ensembling
n_epochs = 100                 # number of epochs to be trained
n_cycles = 20                  # number of models to be generated
snapShotEN = SnapshotEnsemble(n_epochs, n_cycles, 0.01)



[INFO] using 'triangular' method


In [19]:
steps_per_epoch = 24         # number of steps in each epoch

# train the network
print("[INFO] training network...")
H = model.fit(
	x=aug.flow(trainX, trainY, 32),
	validation_data=(testX, testY),
	steps_per_epoch=steps_per_epoch,
	epochs=n_epochs,
	callbacks=[clr,snapShotEN],
	verbose=1)




[INFO] training network...
Epoch 1/100
24/24 [==============================] - 34s 1s/step - loss: 3.5257 - accuracy: 0.0977 - val_loss: 4.3336 - val_accuracy: 0.1062
Epoch 2/100
24/24 [==============================] - 16s 665ms/step - loss: 3.1679 - accuracy: 0.1146 - val_loss: 3.2371 - val_accuracy: 0.1419
Epoch 3/100
24/24 [==============================] - 16s 672ms/step - loss: 2.7506 - accuracy: 0.1927 - val_loss: 2.6129 - val_accuracy: 0.2111
Epoch 4/100
24/24 [==============================] - 16s 666ms/step - loss: 2.3786 - accuracy: 0.2279 - val_loss: 2.2391 - val_accuracy: 0.2802
Epoch 5/100
24/24 [==============================] - 16s 665ms/step - loss: 2.0895 - accuracy: 0.3229 - val_loss: 1.9826 - val_accuracy: 0.3447
>saved snapshot snapshot_model_1.h5, epoch 4
Epoch 6/100
24/24 [==============================] - 16s 683ms/step - loss: 1.8873 - accuracy: 0.3659 - val_loss: 2.0594 - val_accuracy: 0.3868
Epoch 7/100
24/24 [==============================] - 16s 667ms/step

In [25]:
import os


print("[INFO] evaluating network...")
predictions = model.predict(x=testX)
print(classification_report(testY.argmax(axis=1) , predictions.argmax(axis=1) , target_names = ["airplane", "automobile", "bird", "cat", "deer", "dog","frog", "horse", "ship", "truck"]))





[INFO] evaluating network...
              precision    recall  f1-score   support

    airplane       0.36      0.09      0.15      1000
  automobile       0.35      0.82      0.49      1000
        bird       0.45      0.07      0.13      1000
         cat       0.36      0.09      0.15      1000
        deer       0.37      0.28      0.32      1000
         dog       0.33      0.47      0.39      1000
        frog       0.43      0.67      0.53      1000
       horse       0.56      0.49      0.52      1000
        ship       0.43      0.46      0.44      1000
       truck       0.35      0.46      0.40      1000

    accuracy                           0.39     10000
   macro avg       0.40      0.39      0.35     10000
weighted avg       0.40      0.39      0.35     10000



In [26]:
# plot the training and validation accuracy 
N = np.arange(0, n_epochs)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Train and val Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("accuracy.png")


In [27]:
# plot the training and validation loss  
N = np.arange(0, n_epochs)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training and val loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("loss.png")


In [28]:
# plot the learning rate vs training iterations
N = np.arange(0, len(clr.history["lr"]))
plt.figure()
plt.plot(N, clr.history["lr"])
plt.title("Cyclical Learning Rate (CLR)")
plt.xlabel("Training Iterations")
plt.ylabel("Learning Rate")
plt.savefig("clr_plot.png")

In [29]:
# plot learning rate vs number of epochs
import os
perEpoch = np.arange(0, n_epochs)
cyclicLR = []
clr_all_iter = clr.history["lr"]
for ind in range(len(clr_all_iter)):
    if ind % steps_per_epoch==0:
        cyclicLR.append(clr_all_iter[ind])

plt.figure()
plt.plot(perEpoch, cyclicLR)
plt.title("Cyclical Learning Rate (CLR)")
plt.xlabel("Epoch")
plt.ylabel("Learning Rate")
plt.savefig("clr_epochs.png")